In [79]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import re

import warnings
warnings.filterwarnings('ignore')

from flask import Flask
from flask import request
from flask import render_template
import mysql.connector
from mysql.connector import Error
import string
from word2number import w2n

In [93]:
# Function to tokenize the episode names
# def token(s): 
#     getVals = list([val for val in s
#                 if val.isalpha() or val.isnumeric()])

#     result = "".join(getVals)
#     result = result.lower()
#     return result



def token(text):
    # Remove everything except normal alphabets and numbers
    cleaned_text = re.sub(r'[^a-zA-Z0-9]', ' ', text)
    
    # Tokenize the cleaned text
    tokens = cleaned_text.split()
    result =""
    for a in tokens:
        result = result + a
    result = result.lower()
    for i in range(50):
        result = result.replace("pt"+str(i),"part"+str(i))
        result = result.replace("p"+str(i),"part"+str(i))
        result = result.replace("partone","part1")
        result = result.replace("parttwo","part2")
        result = result.replace("partthree","part3")
        result = result.replace("partfour","part4")
        result = result.replace("partfive","part5")
        
    return result

In [94]:
s="farewellôçªpartfive"
token(s)

'farewellpart5'

In [95]:
# Test Data

# v_id = [
# '218d187e-5552-4b5b-bc8f-f32b95f16c18',
# '9594d77b-426a-4304-90b0-8d4e28873e84',
# '4dcf69ae-b336-4c82-8364-e7552f138488',
# '20ad2d10-0626-4270-825e-65d344426329',
# 'eedfef3a-f5b5-474b-a52e-a035af80564c',
# 'e5468efa-51e0-4cda-9c23-9f201e994a41',
# '2fd4780a-e15d-4461-9d47-487bd7b3a5c3',
# '10090965-7c3c-4511-b7e0-bc66f54bdb20',
# '52bd5a7c-d3be-4736-a3c0-6ca87c4bd52f',
# '3682bf04-4b0f-4a8e-be65-e87de06b9ae1',
# 'c42fbdf9-96a6-464a-ac97-0ea3f055d5c4',
# 'c0ab2060-ab20-447d-8ae1-c083b4658db7',
# 'b38a02ec-7995-4c35-8287-52cf2dda1f8b',
# '30f0aa86-af2c-4093-8915-bb66740ab59e',
# '4f3864d0-4f07-4881-b6ae-204ab740ceef',
# '7ac5fc11-f0e8-4705-a772-8ee0d1767af4',
# 'e9fe6383-f293-4463-b04a-bd1faf417df0',
# '39dd7709-5e16-4256-878a-d96e15e4817a',
# '2cb6720e-360c-41f4-9c43-672938a4df3b',
# '6946657e-cbe8-11ee-8591-00a0d1ebeba9',
# '989a85e6-d35d-4c68-adaa-0ce624c82f91',
# 'b0134673-d5d0-4002-9ac6-513e82703e11',
# 'a973d5fc-f868-4e07-bf5b-7790fbd35758',
# 'de38a1b9-b8e3-41c1-9441-1b7ea6bfbd40'
# ]
#Not Mapped
v_id = ['96932f06-f7e1-4aa7-b84d-b96e5b32e999'
'0b83b2e5-c0c9-46ff-8ae0-fd533b6205dc',
'01d695e6-f736-4f93-ac16-bb9d519e4fe5',
'8ae09146-30ad-44b4-acbc-e052ea97cc28',
'06a6d523-7aec-4924-9b30-cdb6ceeaea6d',
'9c299895-feae-4688-b3f1-0c8a06405fb5',
'7ddf4385-f3ac-467a-8277-f4fe2a2b3ce5',
'2d3b088e-e133-4c33-9046-a49aef10cf9b',
'4fa2b166-dff3-4de6-a3f3-22792814bbcd',
'975b0abb-9afc-4c82-9841-7f373812a9b5',
'cd584618-56a9-458e-b5ff-47b87c18b1b6',
'243ae15c-484f-44a1-88cd-3abe578e6609',
'3e02d3ff-75b7-4c6f-a9a1-dd6ebd1b45fc',
'3845a9ff-3155-44f5-aa6c-05975729c985',
'8b6c2530-5917-412d-a5a2-946396a7fd8f',
'8bc01b16-60cb-4e7c-9732-718e38420b63',
'c7beb9d2-4c9c-4e68-befa-2c57bcbe86a2',
'897f9d59-00af-4cf2-b967-129fb92cd2c5',
'7c11be8d-9c4e-4576-9cd2-f56e1c547009',
'3bc16851-d4a0-4bf0-8cb3-cee36b09eb5f',
'1658b889-fcb4-4cb9-be9d-bb9db7cef629',
'f6a12ff9-ebb6-49a9-a665-6ee556a62cc1',
'c9ea4356-ac1e-4167-83d4-c5004839acae'
]
# v_id = ['145bb340-a4b7-4d71-9efe-a5788b8d7975',
#         '23cab86e-a114-4fcb-8885-ed679dcf14a6',
#         'e2cca245-cfbb-4c03-9942-836449ec2fd2',
#        'cbe35079-9c75-4193-b88d-5b6ebe8e17a9',
#        'd8366718-1ca1-4434-88aa-69933ebd8e18']
# v_id = ['96932f06-f7e1-4aa7-b84d-b96e5b32e999',
# #'5008fdb1-0c66-46e7-8ecf-cbe03021f950',
# #'0707deae-17b9-4ec1-b45d-56532f320f86',
# '946fbd54-d3fd-4e66-b5a1-fac18d6305ed',
# '52c87a97-0663-4188-83b0-8d1db0eae6ec',
# '92e510f1-9ce2-478b-a93d-8e40fd0ca529',
# 'f9e1fe71-c91a-4ddb-9371-937e40e18ec9',
# '720b5551-e62b-4e08-80f7-2067e7151673',
# '097c473f-3598-48eb-b303-d822c3fe0206',
# 'e016138e-bb89-4179-82c1-d60e4e647b72',
# '2b6e5d08-758d-4837-abda-b46ba6644bcd',
# # 'c6236338-f69c-44fa-9d41-7c273bea5ab7',
# '7c3cc2de-3ff6-48a2-b508-bd9b00f4a61f',
# '2f9f6b2c-7bb1-48fa-baaf-cfea2ab20602',
# 'ffcb664c-9c70-4e1a-bda7-e8d98382178d',
# '5a6923c7-fffd-4862-b7f8-949bcd504a50',
# 'd8aeab3c-56a9-461a-9fea-82d02a35a1fd',
# 'dd8d7f63-54d2-4968-b5b7-40d8ffe75531',
# 'c0da4f8d-483b-463a-92ef-7ed97f382221',
# '932b94fd-7769-4ebd-8e5c-273e70961c8c',
# '850c08ec-28f2-4feb-bb82-f87ba5b3dfdd',
# '2ec1fe84-92d9-4080-92e4-b52dda7c518b',
# 'cd13c993-7b0a-4113-8cb3-c3bf48f91fc0',
# #'a7beff11-d9e5-42e4-af88-8f96a30fdec8',
# 'ee617385-353d-42d8-83ed-44097c368512']

In [96]:
# Connect to MYSQL Database
connection = mysql.connector.connect(host='winv-ahd-003',
            database='vonly_data_feed_us_staging',
            user='vonly-agent',
            password='a714fded-311c-4215-8b8b-5df4086e264b')

In [98]:
test_case=3
for id_1 in v_id:
    print("\n","Test Case: ",test_case)
    #print(id_1)
    test_case=test_case+1
    
    # SQL Query
    sql_select_Query = """  
    SELECT  te.platform,te.portal,te.vonly_asset_id ,ts.title 'season_name', ts.num 'season_no',ts.portal_item_id 'season_portal_item_id', 
    te.title 'episode_name',te.num 'episode_no' ,te.portal_item_id  'episode_portal_item_id'
    FROM tv_id_mappings tim
    INNER JOIN tv_ids te ON te.id=tim.episode_vonly_id
    INNER JOIN tv_ids ts ON ts.id=tim.season_vonly_id
    WHERE tim.season_vonly_id IN (
    SELECT id FROM tv_ids WHERE scope='tv_season' AND vonly_asset_id='{}');""".format(id_1)   

    cursor = connection.cursor()        
    cursor.execute(sql_select_Query)
    records = cursor.fetchall()
#     for r in records:
#         print(r[0], r[7], r[6])
    
    # Dictionaries for each portal and one general to compare where for each keys is episode number and value is apisode name
    amazon = dict()
    vudu = dict()
    google = dict()
    apple = dict()
    itunes = dict()
    general = dict()
    
    
    # Code to fill the above dictionaries, skipping the trailers
    for r in records:
        platform = r[0]
        episode_no = r[7]
        episode_name = r[6]
        portal = r[1]
        if(platform == 'iTunes'):
            if(int(episode_no)>50 or int(episode_no)==0 or episode_no==None): 
                continue   
            itunes[episode_no] = episode_name
            
        elif(platform == 'VUDU' ):
            if(int(episode_no)>50 or int(episode_no)==0 or episode_no==None): 
                continue
            vudu[episode_no] = episode_name
            
        elif(platform == 'Amazon Prime Video' ):
            if(int(episode_no)>50 or int(episode_no)==0 or episode_no==None): 
                continue
            amazon[episode_no] = episode_name

        elif(platform == 'AppleTVApp' and r[1] == 'iTunes'):
            if(int(episode_no)>50 or int(episode_no)==0 or episode_no==None): 
                continue
            apple[episode_no] = episode_name
            
        elif(platform == 'Google Play' ):
            #print(episode_no)
            if(episode_no==None or int(str(episode_no))>50 or int(str(episode_no))==0): 
                continue
            google[episode_no] = episode_name
            
        #else:
            #print("Portal not in the list")
    
    
    amazon_len,itunes_len,apple_len,vudu_len,google_len=len(amazon),len(itunes),len(apple),len(vudu),len(google)
    print("Amazon Length= ",amazon_len)
    print("iTunes Length= ",itunes_len)
    print("Apple Length= ",apple_len)
    print("Vudu Length= ",vudu_len)
    print("Google Length= ",google_len)
    print("\n")
    
    # Checking if any portal is not present totally
    amazon_class=itunes_class=apple_class=vudu_class=google_class=0
    if(amazon_len == 0 ):
        amazon_class=1
        print("Amazon portal is not there.")

    if(itunes_len == 0):
        itunes_class=1
        print("iTunes portal is not there.")

    if(apple_len == 0):
        apple_class=1
        print("Apple portal is not there.")

    if(vudu_len == 0):
        vudu_class=1
        print("Vudu portal is not there.")

    if(google_len == 0):
        google_class=1
        print("Google portal is not there.")

    
    # Check whether each portal have same number of episodes or not
    x=0
    
    len_= max(amazon_len,itunes_len,apple_len,vudu_len,google_len)
    print("len=",len_)
    
    if(itunes_class==0):
        if(itunes_len == len_):
            x=x+1
        else:
            print("iTunes length is not same")

    if(apple_class==0):
        if(apple_len == len_):
            x=x+1
        else:
            print("Apple length is not same")
    
    if(amazon_class==0):
        if(amazon_len == len_):
            x=x+1
        else:
            print("Amazon length is not same")

    if(google_class==0):    
        if(google_len == len_):
            x=x+1
        else:
            print("Google length is not same")
    
    if(vudu_class==0):
        if(vudu_len == len_):
            x=x+1
        else:
            print("Vudu length is not same")
        


    #print(x) 
#     if(x!=5-(amazon_class+itunes_class+apple_class+vudu_class+google_class)):
#         print("\033[1mNot Properly Mapped Due to Size\033[0m")
#         continue
    
#     print(len(itunes))
#     print(len(apple))
#     print(len(amazon))
#     print(len(vudu))
#     print(len(google))
        
         
#    print("a",amazon,"\n")
#    print("i",itunes,"\n")
#    print("ap",apple,"\n")
#    print("v",vudu,"\n")
    print("g",google,"\n")

    # Fillind the general dict,so that the code is independent of any particular portal
    if(amazon_class==0 and amazon_len==len_):
        general = amazon.copy()
    elif(itunes_class==0 and itunes_len==len_):
        general = itunes.copy()
    elif(apple_class==0 and apple_len==len_):
        general = apple.copy()
    elif(google_class==0 and google_len==len_):
        general = google.copy()
    elif(vudu_class==0 and vudu_len==len_):
        general = vudu.copy()
        
    
#     print(general).

# Check whether the name and corresponding number of each episode is same or not
    y = 0  

# Update the loop range to iterate up to the minimum length
    common_len = min(len(general), apple_len)

    if apple_class == 0:
        for i in range(1, common_len + 1):
            m = token(general[str(i)])

            # Check if the episode number exists in the itunes dictionary
            if str(i) in apple:
                n = token(apple[str(i)])
                if n.find(m) != -1:
                    y += 1
                else:
                    print("The name doesn't match in apple:-")
                    print(f"general:{m} apple:{n}")
            #else:
                #print(f"Episode number {i} not found in apple")

                
#             print(f"general:{m} apple:{n}")
#             print(y)
            
    #print(y)        
    #print("\n")
    common_len = min(len(general), google_len)
    if(google_class==0):
        for i in range(1, common_len + 1):
            m = token(general[str(i)])

            # Check if the episode number exists in the itunes dictionary
            if str(i) in google:
                n = token(google[str(i)])
                if n.find(m) != -1:
                    y += 1
                else:
                    print("The name doesn't match in google:-")
                    print(f"general:{m} google:{n}")
            #else:
                #print(f"Episode number {i} not found in google")
                
#             print(f"general:{m} google:{n}")
#             print(y)
    
    #print(y)
    print("\n")
    common_len = min(len(general), itunes_len)
    if itunes_class == 0:
        for i in range(1, common_len + 1):
            m = token(general[str(i)])

            # Check if the episode number exists in the itunes dictionary
            if str(i) in itunes:
                n = token(itunes[str(i)])
                if n.find(m) != -1:
                    y += 1
                else:
                    print("The name doesn't match in itunes:-")
                    print(f"general:{m} itunes:{n}")
            #else:
                #print(f"Episode number {i} not found in itunes")
            
#             print(f"general:{m} itunes:{n}")
#             print(y)
    
    #print(y)
    print("\n")
    common_len = min(len(general), vudu_len)
    if(vudu_class==0):
        for i in range(1, common_len + 1):
            m = token(general[str(i)])
            
            # Check if the episode number exists in the itunes dictionary
            if str(i) in vudu:
                n = token(vudu[str(i)])
                if n.find(m) != -1:
                    y += 1
                else:
                    print("The name doesn't match in vudu:-")
                    print(f"general:{m} vudu:{n}")
            #else:
               #print(f"Episode number {i} not found in vudu")
                
#             print(f"general:{m} vudu:{n}")
#             print(y)
    
    #print(y)
    print("\n")
    common_len = min(len(general), amazon_len)
    if(amazon_class==0):
        for i in range(1, common_len + 1):
            m = token(general[str(i)])

            # Check if the episode number exists in the itunes dictionary
            if str(i) in amazon:
                n = token(amazon[str(i)])
                if n.find(m) != -1:
                    y += 1
                else:
                    print("The name doesn't match in amazon:-")
                    print(f"general:{m} amazon:{n}")
            #else:
                #print(f"Episode number {i} not found in amazon")
                
#             print(f"general:{m} amazon:{n}")
            #print(y)
            

    
    print("Y should be=",(amazon_len+itunes_len+apple_len+vudu_len+google_len))
    print("Y is=",y)
    print("X should be=",(5-(amazon_class+itunes_class+apple_class+vudu_class+google_class)))
    print("X is=",x)
    
    #  If episode number and corresponding name are same then print properly mapped else vice versa
    if(x==(5-(amazon_class+itunes_class+apple_class+vudu_class+google_class)) and y==(amazon_len+itunes_len+apple_len+vudu_len+google_len)):
        print("\033[1mProperly Mapped\033[0m")
    else:
        print("\033[1mNot Properly Mapped\033[0m")




 Test Case:  3
Amazon Length=  0
iTunes Length=  0
Apple Length=  0
Vudu Length=  0
Google Length=  0


Amazon portal is not there.
iTunes portal is not there.
Apple portal is not there.
Vudu portal is not there.
Google portal is not there.
len= 0
g {} 







Y should be= 0
Y is= 0
X should be= 0
X is= 0
Properly Mapped

 Test Case:  4
Amazon Length=  0
iTunes Length=  25
Apple Length=  25
Vudu Length=  25
Google Length=  7


Amazon portal is not there.
len= 25
Google length is not same
g {'1': 'The Fresh Prince Project', '4': "Not With My Pig, You Don't", '2': 'Bang the Drum, Ashley', '5': 'Homeboy, Sweet Homeboy', '13': 'Knowledge Is Power', '6': 'Mistaken Identity', '3': 'Clubba Hubba'} 







Y should be= 82
Y is= 81
X should be= 4
X is= 3
Not Properly Mapped

 Test Case:  5
Amazon Length=  0
iTunes Length=  0
Apple Length=  0
Vudu Length=  0
Google Length=  0


Amazon portal is not there.
iTunes portal is not there.
Apple portal is not there.
Vudu portal is not there.
Google po

Amazon Length=  0
iTunes Length=  0
Apple Length=  0
Vudu Length=  0
Google Length=  50


Amazon portal is not there.
iTunes portal is not there.
Apple portal is not there.
Vudu portal is not there.
len= 50
g {'7': 'Season 1, Episode 7 Nine Lives', '23': 'Season 2, Episode 8 The Soldier', '33': 'Season 2, Episode 18 Crossties', '10': 'Season 1, Episode 10 Alethea', '48': 'Season 3, Episode 10 Besieged: Death On A Cold Mountain', '39': 'Season 3, Episode 1 Blood Of The Dragon - part 1', '34': 'Season 2, Episode 19 The Passion Of Chen Yi', '16': 'Season 2, Episode 1 The Well', '25': 'Season 2, Episode 10 The Hoots', '1': 'Season 1, Episode 1 King Of The Mountain', '38': 'Season 2, Episode 23 The Cenopath, Part 2', '40': 'Season 3, Episode 2 Blood Of The Dragon - part 2', '24': 'Season 2, Episode 9 The Salamander', '5': 'Season 1, Episode 5 The Tide', '28': 'Season 2, Episode 13 Empty Pages Of A Dead Book', '11': 'Season 1, Episode 11 A Praying Mantis Kills', '2': 'Season 1, Episode 2 Dar

KeyError: '9'